#Import

In [1]:
pip install biopython

     |████████████████████████████████| 2.3MB 3.8MB/s 


In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
import os

In [3]:
import pandas as pd
import numpy as np

#Burrows-Wheeler Transform and FM Index

In [4]:
# Returns list of rotations of input string t
def rotations(t):
    tt = t + t
    return [ tt[i:i+len(t)] for i in range(0, len(t)) ]

# Returns lexicographically sorted list of rot; rot = rotations(t)
def bwm(rot):
    return sorted(rot)

# Returns BWT(bwm_t) by way of the BWM; bwt_t = bwt(rot); rot = rotations(t)
def bwtViaBwm(bwMatrix):
    ''' Given T, returns BWT(T) by way of the BWM '''
    return ''.join(map(lambda x: x[-1], bwMatrix))

###Test

In [5]:
t = 'BANANA$'
rotations(t)

['BANANA$', 'ANANA$B', 'NANA$BA', 'ANA$BAN', 'NA$BANA', 'A$BANAN', '$BANANA']

In [6]:
#bwm(t)

In [7]:
#bwtViaBwm(t)

### Test - new functions

In [8]:
t = 'BANANA$'
r = rotations(t)
r

['BANANA$', 'ANANA$B', 'NANA$BA', 'ANA$BAN', 'NA$BANA', 'A$BANAN', '$BANANA']

In [9]:
bwm_t = bwm(r)
bwm_t

['$BANANA', 'A$BANAN', 'ANA$BAN', 'ANANA$B', 'BANANA$', 'NA$BANA', 'NANA$BA']

In [10]:
bw_transform = bwtViaBwm(bwm_t)
bw_transform

'ANNB$AA'

##BWT

In [11]:
#returns first column of BW matrix

def firstColumn(bwMatrix):
    return ''.join(map(lambda x: x[0], bwMatrix))

In [12]:
###### TEST ######
fc = firstColumn(bwm_t)
fc

'$AAABNN'

In [13]:
#returns FM index C matrix; fc = first column

def fm_c(fc):
  c = []
  n = 0
  ch = fc[0]
  i = 1
  while i < len(fc):
    if fc[i] != ch:
      c.append([ch, n])
      ch = fc[i]
      n = i
    i += 1
  c.append([ch, n])
  return c

In [14]:
###### TEST ######
c_arr = fm_c(fc)
c_arr

[['$', 0], ['A', 1], ['B', 4], ['N', 5]]

In [15]:
#returns first appearance of character c in first column of bwt

def C(c, cArr):
    for i in range(0, len(cArr)):
        if cArr[i][0] == c:
            return cArr[i][1]

In [16]:
#returns element after c in first column matrix

def next(c, cArr):
    for i in range(0, len(cArr)-1):
        if cArr[i][0] == c:
            return cArr[i+1][0]
    return ''

In [17]:
###### TEST ######
print(C('A', c_arr))
print(C('N', c_arr))
print()
print(next('A', c_arr))
print(next('N', c_arr))

1
5

B



In [18]:
def formatSAarray(m):
    arr = []
    for i in range(0, len(m)):
        arr.append(m.index[i])
    return arr

In [19]:
#returns FM suffix array

def suffixArray(rot):
    m = pd.DataFrame(rot)
    m = m.sort_values(by=[0])
    arr = formatSAarray(m)
    return arr

In [20]:
###### TEST ######
suff = suffixArray(r)
suff

[6, 5, 3, 1, 0, 4, 2]

In [21]:
def fm_occ(cArr, BWT):
  occ = []
  for i in range(0, len(cArr)):
    k = cArr[i][0]
    occ_row = []
    z = 0
    for j in range(0, len(BWT)):
        if BWT[j] == k:
            z += 1
        occ_row.append(z)
    occ.append([k, occ_row])
  return occ



In [22]:
###### TEST ######
occ_matrix = fm_occ(c_arr, bw_transform)
occ_matrix

[['$', [0, 0, 0, 0, 1, 1, 1]],
 ['A', [1, 1, 1, 1, 1, 2, 3]],
 ['B', [0, 0, 0, 1, 1, 1, 1]],
 ['N', [0, 1, 2, 2, 2, 2, 2]]]

In [23]:
###### TEST ######
print(occ_matrix[1][1][5])
print(len(occ_matrix[0][1]))

2
7


In [24]:
#returns (c,k) value in Tally matrix

def Occ(c, k, occMatrix):
  if k < len(occMatrix[0][1]):
    for i in range(0, len(occMatrix)):
        if occMatrix[i][0] == c:
            return occMatrix[i][1][k]
  return 

In [25]:
###### TEST ######
print(Occ('A', 2, occ_matrix))
print(Occ('A', 7, occ_matrix))

1
None


In [26]:
#Searching for position of string p in t

def bwt_fm_search(p, t):
    t = t + '$'
    rot = rotations(t)
    bwm_t = bwm(rot)
    bw_transform = bwtViaBwm(bwm_t)
    fc = firstColumn(bwm_t)
    fm_c_arr = fm_c(fc)
    sa = suffixArray(rot)
    fm_occ_matrix = fm_occ(fm_c_arr, bw_transform)

    lastCharacter = p[len(p)-1]
    c_next = next(lastCharacter, fm_c_arr)
    x = 0
    
    if (c_next != ''):
        x = C(c_next, fm_c_arr)
        
    start = C(lastCharacter, fm_c_arr) + 1
    end = x
    
    for i in range(1, len(p)):
        ch = p[len(p)-1-i]
        start = C(ch, fm_c_arr) + Occ(ch, start-1-1, fm_occ_matrix) + 1
        end = C(ch, fm_c_arr) + Occ(ch, end-1, fm_occ_matrix)
    
    return sa[start-1:end]

In [27]:
bwt_fm_search('ANA', 'BANANA')

[3, 1]

### Test

In [57]:
tt = 'TCGAGATCGAGATCGAGATCGAGA'
pp = 'TCGAGA'
bwt_fm_search(pp, tt)
pp_rc = Seq(pp).reverse_complement()
bwt_fm_search(pp_rc, tt)

print(pp_rc)
print(bwt_fm_search(pp, tt))
print(bwt_fm_search(pp_rc, tt))

TCTCGA
[18, 12, 6, 0]
[]


# Global Alignment

In [29]:
# scoring matrix for specific values
def scoringMatrix(a, b):
    if a == b: return 1
    if a == '_' or b == '_' : return -7
    maxb, minb = max(a, b), min(a, b)
    if minb == 'A' and maxb == 'G': return -1
    if minb == 'C' and maxb == 'T': return -1
    return -3

In [30]:
def globalAlignment(x, y, s):
    D = np.zeros((len(x) + 1, len(y) + 1), dtype=int)
    
    for i in range(1, len(x) + 1):
        D[i,0] = D[i-1,0] + s(x[i-1], '_')  
    for j in range(1, len(y)+1):
        D[0,j] = D[0,j-1] + s('_', y[j-1])
    
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            D[i,j] = max(D[i-1,j]   + s(x[i-1], '_'),
                         D[i,j-1]   + s('_', y[j-1]), 
                         D[i-1,j-1] + s(x[i-1], y[j-1]))
            
    # function returns table and global alignment score
    #alignment score is in cell (n,m) of the matrix
    return D, D[len(x),len(y)] 

In [31]:
def traceback(x,y,V,s):
    
    # initializing starting position cell(n,m)
    i=len(x)
    j=len(y)
    
    # initializing strings we use to represent alignments in x, y, edit transcript and global alignment
    ax, ay, am, tr = '', '', '', ''
    
    # exit condition is when we reach cell (0,0)
    while i > 0 or j > 0:
        
        # calculating diagonal, horizontal and vertical scores for current cell
        d, v, h = -100, -100, -100
        
        if i > 0 and j > 0:
            delta = 1 if x[i-1] == y[j-1] else 0
            d = V[i-1,j-1] + s(x[i-1], y[j-1])  # diagonal movement   
        if i > 0: v = V[i-1,j] + s(x[i-1], '_')  # vertical movement
        if j > 0: h = V[i,j-1] + s('_', y[j-1])  # horizontal movement
            
        # backtracing to next (previous) cell
        if d >= v and d >= h:
            ax += x[i-1]
            ay += y[j-1]
            if delta == 1:
                tr += 'M'
                am += '|'
            else:
                tr += 'R'
                am += ' '
            i -= 1
            j -= 1
        elif v >= h:
            ax += x[i-1]
            ay += '_'
            tr += 'D'
            am += ' '
            i -= 1
        else:
            ay += y[j-1]
            ax += '_'
            tr += 'I'
            am += ' '
            j -= 1
            
    alignment='\n'.join([ax[::-1], am[::-1], ay[::-1]])
    return alignment, tr[::-1]

### Test

In [32]:
#x = 'GCGTATGCACGC'
#y = 'GCTATGCCACGC'

x = 'TACGTCAGC'
y = 'TATGTCATGC'

D, alignmentScore = globalAlignment(x, y, scoringMatrix)
print (D)
print('alingmentScore = ', alignmentScore)

[[  0  -7 -14 -21 -28 -35 -42 -49 -56 -63 -70]
 [ -7   1  -6 -13 -20 -27 -34 -41 -48 -55 -62]
 [-14  -6   2  -5 -12 -19 -26 -33 -40 -47 -54]
 [-21 -13  -5   1  -6 -13 -18 -25 -32 -39 -46]
 [-28 -20 -12  -6   2  -5 -12 -19 -26 -31 -38]
 [-35 -27 -19 -11  -5   3  -4 -11 -18 -25 -32]
 [-42 -34 -26 -18 -12  -4   4  -3 -10 -17 -24]
 [-49 -41 -33 -25 -19 -11  -3   5  -2  -9 -16]
 [-56 -48 -40 -32 -24 -18 -10  -2   2  -1  -8]
 [-63 -55 -47 -39 -31 -25 -17  -9  -3  -1   0]]
alingmentScore =  0


In [33]:
a, t = traceback(x, y, D, scoringMatrix)
print(a)
print('edit transcript: ', t)

TACGTCA_GC
|| |||| ||
TATGTCATGC
edit transcript:  MMRMMMMIMM


In [34]:
a, t = traceback(x, y, globalAlignment(x, y, scoringMatrix)[0], scoringMatrix)
print(a)
print('edit transcript: ', t)

TACGTCA_GC
|| |||| ||
TATGTCATGC
edit transcript:  MMRMMMMIMM


# Seed & Extend

In [35]:
# test reverse_complement function
w = 'CACGTTGCTAAG'
w = Seq(w)
q = w.reverse_complement()
q

Seq('CTTAGCAACGTG')

In [36]:
#returns sorted list of tuples for alignment (position, alignment score, edit transcript)

def align(ref, read, length, margin, scorMatrix):
    seed = read[0:length]
    p = bwt_fm_search(seed, ref)
    
    alignList = []

    for i in range (0, len(p)):
      start = p[i]
      end = start + len(read) + margin
      new_ref = ref[start:end]

      D, alignmentScore = globalAlignment(new_ref, read, scorMatrix)
      alignment, transcript = traceback(new_ref, read, D, scorMatrix)
      alignList.append((start, alignmentScore, transcript))

    if (len(alignList) == 0):
      r = Seq(read)
      read_rc = r.reverse_complement()
      seed_rc = read_rc[0:length]
      p_rc = bwt_fm_search(seed_rc, ref)

      for i in range (0, len(p_rc)):
        start = p_rc[i]
        end = start + len(read_rc) + margin
        new_ref = ref[start:end]

        D, alignmentScore = globalAlignment(new_ref, read_rc, scorMatrix)
        alignment, transcript = traceback(new_ref, read_rc, D, scorMatrix)
        alignList.append((start, alignmentScore, transcript))

    alignList.sort(key=lambda tup: tup[1], reverse=True)
    return alignList

### Test

In [37]:
x = 'TTCTGCCCAGCATCCACGAGGCCACCGTCCCCGCCTCCTTGAGATTTGC'
y = 'TGCCCAGCTTC'
length = 5
margin = 3

a = align(x, y, length, margin, scoringMatrix)

print(a)

[(3, -12, 'MMMMMMMMDMDRDM')]


In [38]:
############ TEST ############################

x = 'TCGAGATCGAGATCGAGATCGAGA'
y = 'TCGAGA'
length = 3
margin = 1

a = align(x, y, length, margin, scoringMatrix)

print(a)

[(18, 6, 'MMMMMM'), (12, -1, 'MMMMMMD'), (6, -1, 'MMMMMMD'), (0, -1, 'MMMMMMD')]


# Working with fasta and fastq files



## Get sequences

In [39]:
def readFasta(fasta_file):
  arr = []
  for seq_record in SeqIO.parse(fasta_file, "fasta"):
    arr.append(seq_record.seq)
  return arr[0]


def readFastq(fastq_file):
  arr = []
  for seq_record in SeqIO.parse(fastq_file, "fastq"):
    arr.append(seq_record.seq)
  return arr

### Test

In [40]:
print(readFastq("example_human_Illumina.pe_1.fastq"))

[Seq('ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAA...TGT'), Seq('ATACATGCACTTGACAATGGTCTTTTTACCGTGGGAGCTCCACACAAAGAAAGG...ATC'), Seq('TGTTTCACTTAGGGGAAAATGGATTTGTTGGCCTCAAATAGCTGCTTTATTAGA...TAG'), Seq('AAGTGAATGAATGTCCTAAAATCTACACTTGTGGCAGGGTCTTCCCTTCCAAAC...AGC'), Seq('GAGTTTGGCCTAAGTGAATGAATGTCCTAAAATCTACACTTGTGGCAGGGTCTT...TTT'), Seq('TGGTGAAATTTCAGGAACCATAGCCATTGAAATGGATGAGGGAACCTATATACA...TTT'), Seq('AGGAACCATAGCCATTGAAATGGATGAGGGAACCTATATACATGCACTCGACAA...GCT'), Seq('CAGACACAAACCTTTCTTTGTGTGGAGCTCCCACGGTAAAAAGACCATTGTCAA...CTC'), Seq('AAATGTAACATTAATTTTTATGCTCTAATTATAAGAACGAAGGGCATTTTAGAA...TTA'), Seq('ACTTTCTATTAAACTGACAGGAAGTTTTTATAAACAAAAACAGCACTTACATTT...GTT'), Seq('TCCAGACACAAACCTTCTTTGTGTGGAGCTCCCAGGGTAAAAAGACCATTGTCG...CCT'), Seq('TTTCACCAAAGTTTGTTACTGTTCACCAGATTCCTAAAAAATAAAATTGATATT...TTG'), Seq('GCTCCCAGGGTAAAAAGACCATTGTCGAGTGCATGTATATAGGTTCCCTCATCC...CTG'), Seq('CTTAGGGGAAAATGGCTTTGTTGGCCTCAAATAGCTGCTTTATTAGATGCAATG...AAA'), Seq('ATTTATTCCAATTTAATGTATTATACTA

In [41]:
print(readFastq("example_human_Illumina.pe_1.fastq")[0])

ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT


In [42]:
print(readFasta("example_human_reference.fasta"))

ATATGTGAAAAGCTTTGTTATGCATTGTGAGACAACACAGTGGGAATATTTCATCATCTGCCTAAGGTTTAAAAGGAAATAACTTTAAGCATGTGTCTAAATAGCAAGTAATGTTTTAGAGCGGATTCTCTTAAATTCAGCTTGGGCGTCTGCAGCATATACACAGCTTGAGCTGTAACCTGACATAGAGACAGGCAACTTCAGTGCCCACTGTTCCTAGGATCCACGGCTTTTTCACACCTAAAACCCCTGAGTGGCACTGTTAAGTATTATGTTATGTTACTTTAGTCATTAAACGTATAAGCATACCTCCAAAGGTTGAATGTAGGCCACTTGCAGAAAGTAGGCAGAATGCTCACATTTAATTCTTGATGATACTGTGTTTAGCTTTCTTATTCTTTGAAATATCATTGAGAAGAAATACTGGCATCTGCTCAAAGTAATTTCTTTTTCAGTTGACAATATTATAAGTAATGTTATTGTATCATTTCCCTACTTGGACAGAGTGTGAAAATTTTGAGGATTGTCTGCCACAAATTTCTTCTTCATTTGCAAAACATTCATGAGATATTATATTTAAATGATTTTATTTAATATTAAGTGTACTTGGTGAAGGTGGCATAGAAAATACAAAATAAAACTAATTTAAAAATATTAACTATTACATTTATAAGAAAGACTTGCTAATCATAACACTGTTCATAATGATTCTGAATAAAGCATTATTTCTTTTACTGAAAACAATTGTAGCTATAACTCAATCATCTAAATTGTCATTAGTTTTATTTCTTTCTAATCGCCTCTAGCTGAAATTTTAATTTTGATTAATTTTCTTTTTCTCCATTGGTTTCGTGTGTGTGTGGAGGTAAAATATACAGAATGTAGAATTTGTTAGTTTTTCTATTTTTACGTTTACACTTCAGTGGCATTTAAATACATGCACCATTTTACCTTCCCACCAGCGTTGCACAAGGTTTCAGTTTCTCCACATCCTGCCCAA

In [43]:
# align function test for human reference fasta and first read from fastq file: (1m 33s) - old implementation, (35s) - new implementation
ref = readFasta("example_human_reference.fasta")
read = readFastq("example_human_Illumina.pe_1.fastq")[0]

alignment = align(ref, read, 10, 2, scoringMatrix)

print(alignment)

[(5793, 62, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDD')]


# Main program improved

#### BWT and aligner for array of reads with same reference

In [51]:
def bwt_fm_search_array(p, fm_c_arr, sa, fm_occ_matrix):
    #t = t + '$'
    #rot = rotations(t)
    #bwm_t = bwm(rot)
    #bw_transform = bwtViaBwm(bwm_t)
    #fc = firstColumn(bwm_t)
    #fm_c_arr = fm_c(fc)
    #sa = suffixArray(rot)
    #fm_occ_matrix = fm_occ(fm_c_arr, bw_transform)

    lastCharacter = p[len(p)-1]
    c_next = next(lastCharacter, fm_c_arr)
    x = 0
    
    if (c_next != ''):
        x = C(c_next, fm_c_arr)
        
    start = C(lastCharacter, fm_c_arr) + 1
    end = x
    
    for i in range(1, len(p)):
        ch = p[len(p)-1-i]
        start = C(ch, fm_c_arr) + Occ(ch, start-1-1, fm_occ_matrix) + 1
        end = C(ch, fm_c_arr) + Occ(ch, end-1, fm_occ_matrix)
    
    return sa[start-1:end]

In [52]:
def align_read_array(ref, read, length, margin, scorMatrix, fm_c_arr, sa, fm_occ_matrix):
    seed = read[0:length]
    p = bwt_fm_search_array(seed, fm_c_arr, sa, fm_occ_matrix)
    
    alignList = []

    for i in range (0, len(p)):
      start = p[i]
      end = start + len(read) + margin
      new_ref = ref[start:end]

      D, alignmentScore = globalAlignment(new_ref, read, scorMatrix)
      alignment, transcript = traceback(new_ref, read, D, scorMatrix)
      alignList.append((start, alignmentScore, transcript))

    if (len(alignList) == 0):
      r = Seq(read)
      read_rc = r.reverse_complement()
      seed_rc = read_rc[0:length]
      p_rc = bwt_fm_search_array(seed_rc, fm_c_arr, sa, fm_occ_matrix)

      for i in range (0, len(p_rc)):
        start = p_rc[i]
        end = start + len(read_rc) + margin
        new_ref = ref[start:end]

        D, alignmentScore = globalAlignment(new_ref, read_rc, scorMatrix)
        alignment, transcript = traceback(new_ref, read_rc, D, scorMatrix)
        alignList.append((start, alignmentScore, transcript))

    alignList.sort(key=lambda tup: tup[1], reverse=True)
    return alignList

#### Main

In [58]:
def mainProgram(fasta_file, fastq_file, seed_length, margin, scoringMatrix):
  ref = readFasta(fasta_file)
  readArray = readFastq(fastq_file)
  alignList = []

  t = ref + '$'
  rot = rotations(t)
  bwm_t = bwm(rot)
  bw_transform = bwtViaBwm(bwm_t)
  fc = firstColumn(bwm_t)
  fm_c_arr = fm_c(fc)
  sa = suffixArray(rot)
  fm_occ_matrix = fm_occ(fm_c_arr, bw_transform)

  for i in range(0, len(readArray)):
    aln = align_read_array(ref, readArray[i], seed_length, margin, scoringMatrix, fm_c_arr, sa, fm_occ_matrix)
    alignList.append(aln)

  return alignList

In [59]:
#scoringMatrix values: march 2, mismatch -2, gap -7
def sMatrix(a, b):
    if a == b: return 2
    if a == '_' or b == '_' : return -5
    return -2

In [60]:
example = mainProgram("example_human_reference.fasta", "example_human_Illumina.pe_1.fastq", 10, 2, sMatrix)

for i in range(0, len(example)):
  print(example[i])

[(5793, 142, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDD')]
[(3203, 122, 'MMMMMMMMMMMRMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMIMMMMMMMMMMMMMMMMMMMMMMMMMDMDD'), (933, -54, 'MMMMMMMMMMRRRRRRMRRRMMRRRRRMRMMDMMDRRMRRRRRRMMRRRRRRRRRRRMRRMMRMRRRRMRRRRRRRMR')]
[(8215, 138, 'MMMMMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDD')]
[(3239, 130, 'MMMMMMMMMMMMMMMRMMMMMMMMRMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDD')]
[]
[(3154, 142, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDD')]
[(3166, 138, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMDD')]
[(3191, 122, 'MMMMMMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMIMMMMMMMMMMMMMDMDD')]
[(9854, 138, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMRMMMMMMMMDDMMM')]
[(5959, 134, 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMRMMMMMMMMMMMMMMMMMMMMMMMMMMMMMRMMMMMMDDM')]
[(3192, 142, 'MMMMMMMMMMMMMMMMMMMMMMM